# Filtrando dígitos 1 e 5 e treinando modelos

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
train1x5 = train_redu[train_redu["label"].isin([1, 5])].copy()
test1x5  = test_redu[test_redu["label"].isin([1, 5])].copy()

train1x5["y"] = train1x5["label"].apply(lambda x: 1 if x == 1 else -1)
test1x5["y"]  = test1x5["label"].apply(lambda x: 1 if x == 1 else -1)

X_1x5 = train1x5[["intensidade", "simetria"]].values.astype(float)
y = train1x5["y"].values.astype(int)

X_train_1x5, X_val_1x5, y_train, y_val = train_test_split(
    X_1x5, y, test_size=0.2, random_state=42, stratify=y
)

X_test_1x5 = test1x5[['intensidade','simetria']].values.astype(float)
y_test = test1x5['y'].values.astype(int)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_1x5)
X_val   = scaler.transform(X_val_1x5)
X_test  = scaler.transform(X_test_1x5)

print(X_train)
print(X_test)

print("Tamanho treino:", X_train.shape)
print("Tamanho validação:", X_val.shape)
print("Tamanho teste:", X_test.shape)